In [ ]:
import random, math
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn import datasets
import functools
from IPython.display import display, clear_output
from numba import jit
random.seed(12345)

eps = 10e-15

Para cada experimento se realizarán diferentes casos de prueba con los cuales se evaluará no solo el rendimiento si no también la precisión de los resultados

In [ ]:
# Generar txt y guardar instancia
def save_instanceA(dataset, instance_name, matriz): 
    if(not os.path.isdir(F"instancias/{dataset}")):
        os.mkdir(F"instancias/{dataset}")
                
    with open(F"instancias/{dataset}/{instance_name}.txt", "w") as f:
        for i in range(0, len(matriz)):
            for j in range(0, len(matriz[i])):
                print(matriz[i][j], file=f, end=" ")
            print(file=f, end="\n")
            
# Generar txt y guardar instancia
def save_instanceB(dataset, instance_name, vector): 
    if(not os.path.isdir(F"instancias/{dataset}")):
        os.mkdir(F"instancias/{dataset}")
                
    with open(F"instancias/{dataset}/{instance_name}.txt", "w") as f:
        for i in range(0, len(vector)):
            print(vector[i], file=f, end=" ")
        print(file=f, end="\n")

In [ ]:
@jit
def guardar_CSV_normal(folder_name, arr_instancias):
    pd.DataFrame(arr_instancias, columns=["dataset", "archivo-A","archivo-b", "n", "notas"]).to_csv(F"instancias/{folder_name}/indice.csv", index=False, header=True)
    
@jit
def guardar_CSV_tridg(folder_name, arr_instancias):
    pd.DataFrame(arr_instancias, columns=["dataset", "archivo", "n", "type", "notas"]).to_csv(F"instancias/{folder_name}/indice.csv", index=False, header=True)

## Matrices Completas

### Genero los "b"

In [ ]:
@jit
def generate_b(n):
    b = np.random.rand(n)
    return b

@jit
def generate_b_dec(n):
    return [rnd() for _ in range(n)]

In [ ]:
@jit
def construir_matriz_completa(a,b,c):
    n = len(b)
    A = np.zeros((n,n))
    for i in range(0, len(b)):
        A[i][i] = b[i]
        if i >= 1:
            A[i][i-1] = a[i]
        if i < len(b)-1:
            A[i][i+1] = c[i]
            
    return A

In [ ]:
@jit
def rnd():
    exp = np.random.randint(-19, -1)
    significand = 0.9 * np.random.random() + 0.1
    return significand * 10**exp

## Matrices completas

### Matrices EDD

In [ ]:
@jit
def edd(n, max_val):
    A = np.random.randint(max_val, size=(n, n))
    for i in range(0, n):
        value = 0;
        for j in range(0, n):
            if i != j:
                value = value + np.abs(A[i][j]);
        A[i][i] = sum_val*n
    return A

### Matrices SDP (Simétricas Definidas Positivas)

In [ ]:
@jit
def sdp(n):
    A = sklearn.datasets.make_spd_matrix(n)
    return A

### Matrices Aleatorias

In [ ]:
@jit
def random(n, nmin, nmax):
    L = np.tril(np.random.randint(max_val, size=(n, n)))
    U = np.triu(np.random.randint(max_val, size=(n, n)))
    for i in range(0, n):
        L[i][i] = 1
    return (L @ U)

## Matrices tridiagonales

### Matrices EDD

In [ ]:
@jit
def tr_edd(n):
    a = np.random.rand(n)
    b = np.zeros(n)
    c = np.random.rand(n)
    for i in range(0, len(a)):
        sum_val = np.abs(a[i]) + np.abs(c[i])
        b[i] = sum_val*n
        
    a[0] = 0
    c[-1] = 0
    A = [a, b, c]
    return A

### Matrices SDP (Simétricas Definidas Positivas)

In [ ]:
@jit
def tr_simetric_definite_positive(n):
    A = sklearn.datasets.make_spd_matrix(n)
    
    a = np.zeros(n)
    b = np.zeros(n)
    c = np.zeros(n)
    
    for i in range(0, len(A)):
        b[i] = A[i][i]
    
    for i in range(1, len(A)):
        a[i] = A[i][i-1]
    
    for i in range(0, len(A)-1):
        c[i] = A[i][i+1]
    
    B = [a, b, c]
    
    return B

### Matrices Aleatorias

In [ ]:
@jit
def tr_random_matrix(n, nmin, nmax):

    A = np.random.randint(nmin, nmax, size=n)
    B = np.random.randint(nmin, nmax, size=n)
    C = np.random.randint(nmin, nmax, size=n)

    L = np.zeros((n, n))
    for i in range(0, n):
        L[i][i] = 1

    for i in range(1, n):
        L[i][i-1] = A[i-1]

    U = np.zeros((n, n))
    for i in range(0, n):
        U[i][i] = B[i]

    for i in range(0, n-1):
        U[i][i+1] = C[i]

    M = L @ U
    
    B = np.diagonal(M)
    
    for i in range(1, n):
        A[i] = M[i][i-1] 

    for i in range(0, n-1):
        C[i] = M[i][i+1]
    
    return [A, B, C]

## Matrices Tridiagonales - 4a

In [ ]:
caso="tr_edd";
arr_instancias = []
for i in range(1, 31):
    clear_output(wait=True)
    display('Instancia: ' + str(i))
    n = 100*i
    A = tr_edd(n)
    b = generate_b(n)
    save_instanceA(caso, F"{caso}-{i}-A", A)
    save_instanceB(caso, F"{caso}-{i}-b", b)
    arr_instancias.append([caso, F"{caso}-{i}-A.txt", F"{caso}-{i}-b.txt", n, ""])
guardar_CSV_normal(caso,arr_instancias)

In [ ]:
caso="tr_sdp";
arr_instancias = []
for i in range(1, 31):
    clear_output(wait=True)
    display('Instancia: ' + str(i))
    n = 100*i
    A = tr_simetric_definite_positive(n)
    b = generate_b(n)
    save_instanceA(caso, F"{caso}-{i}-A", A)
    save_instanceB(caso, F"{caso}-{i}-b", b)
    arr_instancias.append([caso, F"{caso}-{i}-A.txt", F"{caso}-{i}-b.txt", n, ""])
guardar_CSV_normal(caso,arr_instancias)

In [ ]:
caso="tr_random";
arr_instancias = []
for i in range(1, 31):
    clear_output(wait=True)
    display('Instancia: ' + str(i))
    n = 100*i
    nmin = np.random.randint(1, 100)
    nmax = np.random.randint(101, 1000)
    A = tr_random_matrix(n, nmin, nmax)
    b = generate_b(n)
    save_instanceA(caso, F"{caso}-{i}-A", A)
    save_instanceB(caso, F"{caso}-{i}-b", b)
    arr_instancias.append([caso, F"{caso}-{i}-A.txt", F"{caso}-{i}-b.txt", n, ""])
guardar_CSV_normal(caso,arr_instancias)

## Matrices Tridiagonales - 4b

In [ ]:
# Matrices EDD
caso="tr_edd_l";
arr_instancias = []
n = 5000
A = tr_edd(n)
save_instanceA(caso, F"{caso}-A", A)
for j in range(0, 1001):
    b = generate_b(n)
    save_instanceB(caso, F"{caso}-{j}-b", b)
    arr_instancias.append([caso, F"{caso}-{j}-b.txt", n, "b", ""])
arr_instancias.append([caso, F"{caso}-A.txt", n, "A", ""])
guardar_CSV_tridg(caso, arr_instancias)

In [ ]:
# Matrices SDP
caso="tr_sdp_l";
arr_instancias = []
n = 5000
A = tr_simetric_definite_positive(n)
save_instanceA(caso, F"{caso}-A", A)
for j in range(0, 1001):
    b = generate_b(n)
    save_instanceB(caso, F"{caso}-{j}-b", b)
    arr_instancias.append([caso, F"{caso}-{j}-b.txt", n, "b", ""])
arr_instancias.append([caso, F"{caso}-A.txt", n, "A", ""])
guardar_CSV_tridg(caso, arr_instancias)

In [ ]:
# Matrices Random
caso="tr_random_l";
arr_instancias = []
n = 5000
nmin = np.random.randint(1, 100)
nmax = np.random.randint(101, 1000)
A = tr_random_matrix(n, nmin, nmax)
save_instanceA(caso, F"{caso}-A", A)
for j in range(0, 1001):
    b = generate_b(n)
    save_instanceB(caso, F"{caso}-{j}-b", b)
    arr_instancias.append([caso, F"{caso}-{j}-b.txt", n, "b", ""])
arr_instancias.append([caso, F"{caso}-A.txt", n, "A", ""])
guardar_CSV_tridg(caso, arr_instancias)

### Matrices de Prueba

In [ ]:
caso = "tr_pruebas_sin_pivot"
arr_instancias = []

# Sin triangular
A = np.array([[1,2,3,4], [1,4,9,16], [1,8,27,64], [1, 16, 81, 256]])
b = np.array([2,10,44,190])

save_instanceA(caso, F"completa-normal-1-A", A)
save_instanceB(caso, F"completa-normal-1-b", b)
arr_instancias.append([caso, F"completa-normal-1-A.txt", F"completa-normal-1-b.txt", 4, "Matriz Normal Sin Triangular"])

# Sin triangular
A = np.array([[2,1,-1,3], [-2,0,0,0], [4,1,-2,4], [-6,-1,2,-3]])
b = np.array([13,-2,24,-10])

save_instanceA(caso, F"completa-normal-2-A", A)
save_instanceB(caso, F"completa-normal-2-b", b)
arr_instancias.append([caso, F"completa-normal-2-A.txt", F"completa-normal-2-b.txt", 4, "Matriz Normal Sin Triangular"])

# Triangulado
A = np.array([[2,1,-1,3], [0,1,-1,3], [0,0,-1,1], [0,0,0,1]])
b = np.array([13,11,9,16])

save_instanceA(caso, F"triangulada-A", A)
save_instanceB(caso, F"triangulada-b", b)
arr_instancias.append([caso, F"triangulada-A.txt", F"triangulada-b.txt", 4, "Matriz Triangulada"])

# Sin triangular - Decimales chicos
A = np.array([[0.0000002,1,-1,3], [-0.0000002,0,0,0], [0.0000004,1,-2,4], [-0.0000006,-1,2,-3]])
b = np.array([13,-2,24,-10])

save_instanceA(caso, F"sin-triangular-decimales-A", A)
save_instanceB(caso, F"sin-triangular-decimales-b", b)
arr_instancias.append([caso, F"sin-triangular-decimales-A.txt", F"sin-triangular-decimales-b.txt", 4, "Matriz Con Decimales Chicos sin Triangular"])

# Matriz con decimales no tan bajos
A = np.array([[0.003, 0.217], [0.277, 0.138]])
b = np.array([0.437,0.553])

save_instanceA(caso, F"decimales-no-tan-bajos-A", A)
save_instanceB(caso, F"decimales-no-tan-bajos-b", b)
arr_instancias.append([caso, F"decimales-no-tan-bajos-A.txt", F"decimales-no-tan-bajos-b.txt", 2, "Matriz "])

# Falla al intentar triangular porque no hay pivot
A = np.array([[0, 0, 0, 0, 0, 0], [0, -1, 0, 0, 0, -0.08], [0, 1, -1, 0, 1, 0], [0, 0, 1, -1, 0, 0.02], [1, 0, 0, 1, 0, 0.05], [0, 0, 0, 0, 0, 0.05]])
b = np.array([0,0,0,0,0,0])

save_instanceA(caso, F"falla-triangular-A", A)
save_instanceB(caso, F"falla-triangular-b", b)
arr_instancias.append([caso, F"falla-triangular-A.txt", F"falla-triangular-b.txt", 6, "Matriz Permutada"])
guardar_CSV_normal(caso,arr_instancias)

In [ ]:
caso = "tr_pruebas_pivot"
arr_instancias = []

# Canónico
A = np.array([[0, 1, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0],[0, 0, 1, 0, 0, 0],[0, 0, 0, 0, 0, 1],[0, 0, 0, 1, 0, 0],[0, 0, 0, 0, 1, 0]])
b = np.array([1,2,3,4,5,6])

save_instanceA(caso, F"pivot-canonico-A", A)
save_instanceB(caso, F"pivot-canonico-b", b)
arr_instancias.append([caso, F"pivot-canonico-A.txt", F"pivot-canonico-b.txt", 6, "Matriz Permutada"])

# Pivot con Decimales
A = np.array([[0.003000, 59.14],[5.291, -6.130]])
b = np.array([59.17, 46.78])

save_instanceA(caso, F"pivot-decimales-1-A", A)
save_instanceB(caso, F"pivot-decimales-1-b", b)
arr_instancias.append([caso, F"pivot-decimales-1-A.txt", F"pivot-decimales-1-b.txt", 2, "Solucion: x1: 10.00 x2: 1.000"])

# Pivot con Decimales - 2
A = np.array([[30.00, 591400],[5.291, -6.130]])
b = np.array([591700, 46.78])

save_instanceA(caso, F"pivot-decimales-2-A", A)
save_instanceB(caso, F"pivot-decimales-2-b", b)
arr_instancias.append([caso, F"pivot-decimales-2-A.txt", F"pivot-decimales-2-b.txt", 2, "Solucion: x1: -10.00 x2: 1.001"])


# Canónico - Sin Solucion
A = np.array([[1, 1],[2, 2]], dtype="f")
b = np.array([2,5], dtype="f")

save_instanceA(caso, F"sin-sol-A", A)
save_instanceB(caso, F"sin-sol-b", b)
arr_instancias.append([caso, F"sin-sol-A.txt", F"sin-sol-b.txt", 2, "Sin Solucion"])

# Canónico - Solucion infinita
A = np.array([[1, 1],[2, 2]])
b = np.array([2,4])

save_instanceA(caso, F"sol-inf-A", A)
save_instanceB(caso, F"sol-inf-b", b)
arr_instancias.append([caso, F"sol-inf-A.txt", F"sol-inf-b.txt", 2, "Solucion Infinita"])

# Sin triangular - Decimales chicos
A = np.array([[0.0000000000000002,1,-1,3], [2,0.0000000000000002,0,0], [4,1,-2,4], [-6,-1,2,-3]])
b = np.array([13,-2,24,-10])

save_instanceA(caso, F"decimales-A", A)
save_instanceB(caso, F"decimales-b", b)
arr_instancias.append([caso, F"decimales-A.txt", F"decimales-b.txt", 4, "Con decimales chicos y posible error de calculo"])

# Error de calculo por aproximación de decimales
A = np.array([[2,1,-1,3], [-2,0,0,0], [4,1,-2,4], [-6,-1,2,-3]], dtype="f")
b = np.array([13,-2,24,-10], dtype="f")

save_instanceA(caso, F"err-aprox-1-A", A)
save_instanceB(caso, F"err-aprox-1-b", b)
arr_instancias.append([caso, F"err-aprox-1-A.txt", F"err-aprox-1-b.txt", 4, "Matriz con Enteros"])

A = np.array([[2,eps,-1,3], [-2,eps,0,0], [4,1,eps,4], [-6,-1,2,eps]])
b = np.array([13,-2,24,-10])

save_instanceA(caso, F"err-aprox-2-A", A)
save_instanceB(caso, F"err-aprox-2-b", b)
arr_instancias.append([caso, F"err-aprox-2-A.txt", F"err-aprox-2-b.txt", 4, "Matriz con Decimales - Error de calculo en solucion: -0.012490009999999607"])

# Instancia para matrices con diferencia en decimales
for q in range(0, 3):
    A = np.zeros((5,5))
    b = generate_b_dec(5)
    
    vector = [rnd() for _ in range(5)]
    A_AUX  = np.array([vector, vector, vector, vector, vector])

    save_instanceA(caso, F"err-numerico-{q}-A", A)
    save_instanceB(caso, F"err-numerico-{q}-b", b)
    arr_instancias.append([caso, F"err-numerico-{q}-A.txt", F"err-numerico-{q}-b.txt", 5, "A y B con numeros muy chicos"])
guardar_CSV_normal(caso,arr_instancias)